# Training

> ## Checking model.py

In [25]:
!pygmentize pytorch_source/model.py

import torch
import torchvision.models as models

#importing pretrained ResNet for transfer learning
ResNetTransfer = models.resnet50(pretrained=True) 


> ## Checking train.py

In [26]:
!pygmentize pytorch_source/train.py

import argparse
import json
import os
import pandas as pd
import numpy as np

import torch
import torch.optim as optim
import torch.nn as nn
import torch.utils.data

from torchvision import datasets
import torchvision.transforms as transforms

import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau

# imports the model in model.py by name
from model import ResNetTransfer


def model_fn(model_dir):
    """Load the PyTorch model from the `model_dir` directory."""
    print("Loading model.")

    # First, load the parameters used to create the model.
    model_info = {}
    model_info_path = os.path.join(model_dir, 'model_info.pth')
    with open(model_info_path, 'rb') as f:
        model_info = torch.load(f)

    print("model_info: {}".format(model_info))

    # Determine the device and construct the model.
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print("Using device {}.".format(device))

    model = ResNetTransfer

    #fre

> ### Lookup for training and testing data into s3
Instead of loading data and using input_data after that, the s3 bucket where the data was stored previously is read directly

In [3]:
import boto3
import sagemaker

# S3 bucket to load
bucket = "sagemaker-eu-central-1-515611759963"

prefix = "food-classifier"

train_test_folder = 's3://{}/{}/'.format(bucket, prefix)

# session and role
sagemaker_session = sagemaker.Session(default_bucket=bucket)
role = sagemaker.get_execution_role()

train_test_folder

's3://sagemaker-eu-central-1-515611759963/food-classifier/'

> ## Create pytorch estimator

In [5]:
# import a PyTorch wrapper
from sagemaker.pytorch import PyTorch, PyTorchModel

In [7]:
# specify an output path
# prefix is specified above
output_path = 's3://{}/{}'.format(bucket, prefix)

# instantiate a pytorch estimator
estimator = PyTorch(entry_point='train.py',
                    source_dir='pytorch_source', 
                    role=role,
                    framework_version= '1.1.0', #'1.3.1',
                    train_instance_count=1,
                    train_instance_type='ml.p2.xlarge',
                    output_path=output_path,
                    sagemaker_session=sagemaker_session,
                    hyperparameters={
                        'n_classes': nc + 1,  # num of classes for the fully connected layer at the end of the network (defined on the first cells and increased by 1 for pytorch counting)
                        'n_epochs': 3,
                        'img_short_side_resize':256,
                        'img_input_size':224,
                        'num_workers':16,
                        'batch_size':64
                    })

In [8]:
#estimator.fit({'training': input_data})

estimator.fit({'training': train_test_folder}) #to use already loaded data

2020-03-08 17:17:19 Starting - Starting the training job...
2020-03-08 17:17:21 Starting - Launching requested ML instances...
2020-03-08 17:18:18 Starting - Preparing the instances for training.........
2020-03-08 17:19:37 Downloading - Downloading input data......
2020-03-08 17:20:40 Training - Downloading the training image..bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2020-03-08 17:21:00,643 sagemaker-containers INFO     Imported framework sagemaker_pytorch_container.training
2020-03-08 17:21:00,668 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2020-03-08 17:21:01,293 sagemaker_pytorch_container.training INFO     Invoking user training script.
2020-03-08 17:21:01,512 sagemaker-containers INFO     Module train does not provide a setup.py. 
Generating setup.py
2020-03-08 17:21:01,512 sagemaker-containers INFO     Generating setup.cfg
2020-03-08 17:21:01,512 sagemaker-container

> ## Deploy

In [9]:
#predictor = estimator.deploy(instance_type='ml.p2.xlarge', initial_instance_count=1)

-------------!

> ## Delete endpoint

In [10]:
#predictor.delete_endpoint()